In [ ]:
from IPython.display import Image
Image(filename='Lock-and-key.JPG')

1) Import all features from pysb.

In [ ]:
from pysb import * # Model, Monomer, Parameter, Initial, Rule, Observable
from pysb.macros import *

2) Create an instance of the model class

In [ ]:
Model()

3) Create a monomer named enzyme that only has one binding site.

In [ ]:
Monomer('enzyme', ['binding1'])

4) Create a monomer name protein that has two sites. One site is the binding site (Remember: for sites that only take part in bond formation the state can be omitted), the other is the protein state site. The protein state site has two sites: substrate and product.

In [ ]:
Monomer('protein', ['binding', 'state'], {'state': ['sub','pro']})

5) Define a k_forward, k_reverse and k_catalysis parameter with the values 0.003, 0.001 and 0.002 respectively.

In [ ]:
Parameter('kf',0.003)
Parameter('kr',0.001 )
Parameter('kc',0.002 )

6) Define a binding rule where the unbound enzyme binds the protein in state substract

`

In [ ]:
Rule('binding', enzyme(binding1=None) + protein(state='sub', binding=None)
     <> enzyme(binding1=1) % protein(state='sub', binding=1), kf,kr)


7) Create a catalysis rule where the complex dissociates and the protein change state to product and the enzyyme unaltered

    Rule('catalysis', complex >> enzyme + protein in state product, catalysis_rate)

In [ ]:
Rule('dissociation', enzyme(binding1=1)%protein(state='sub',binding=1) 
     >> enzyme(binding1=None)+protein(state='pro', binding=None), kc)

8) Define the initial conditions for the two proteins using the parameters.

In [ ]:
Parameter('enzyme_0', 100)
Parameter('protein_0',50)
Initial(enzyme(binding1=None), enzyme_0 )
Initial(protein(binding=None, state='sub') , protein_0)

9) Define the observables: total enzyme, product and the complex

In [ ]:
Observable('e_total', enzyme())
Observable('e_free', enzyme(binding1=None))
Observable('substrate', protein(binding=None))
Observable('product', protein(binding=None, state='pro'))
Observable('complex', enzyme(binding1=1) % 
           protein(binding=1, state='sub'))

To run the simulation

In [ ]:
%matplotlib inline
from pysb.integrate import odesolve
import matplotlib.pyplot as plt
import numpy as np
tspan = np.linspace(0,100,100)
y = odesolve(model,tspan)
plt.plot(tspan, y['substrate'], label="substrate")
plt.plot(tspan, y['e_total'], label="e_total")
plt.plot(tspan, y['e_free'], label="e_free")
plt.plot(tspan, y['product'], label="product")
plt.plot(tspan, y['complex'], label="complex")
plt.xlabel('time')
plt.ylabel('population')
plt.ylim(0,110)
plt.legend(loc=0)
plt.show()
